### Custom libs to be installed:

In [1]:
# !pip install rgf_python
# !pip install xgboost
# !pip install lightgbm

In [ ]:
import os
import sys
import glob
import pathlib
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from rgf.sklearn import RGFClassifier
from sklearn.ensemble import RandomForestClassifier

/Users/elejke/anaconda3/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [ ]:
sys.path.append(os.path.abspath('../src/'))
    
from data import prepare_submission
from data import data_generator_test
from data import data_generator_train

from model_selection import cross_val_score

from quality_estimator import BaseQualityEstimator

## Data preparation:

In [ ]:
X, Xy, y = data_generator_train(root_dir="../data/ellipse/")

In [ ]:
X_test, Xy_test = data_generator_test(root_dir="../data/ellipse/")

## Algorithm initialization:

In [ ]:
# a number of 1-st stage meta-learners:
lgbm = LGBMClassifier(max_depth=2)
rgf = RGFClassifier(max_leaf=400, algorithm="RGF_Sib", test_interval=100, verbose=False)
rf = RandomForestClassifier(n_estimators=100)
xgb = XGBClassifier(max_depth=2, n_estimators=10)

# QualityEstimator definition with metrics family names to be used for feature search:
q_clf = BaseQualityEstimator(metrics=["dice_coefficient", "mae", "mse"], 
                             unary_metrics=[],
                             matching_metrics=[
                                 "match2tpr",
                                 "match2predBinRate",
                                 "match2gtBinRate"
                             ],
                             meta_clfs={'lgbm': lgbm, 
                                        'rgf': rgf,
                                        'rf': rf, 
                                        'xgb': xgb})

# Algorithm fitting 

* 1-st stage metrics: generation in the search space
* 2-nd stage metrics: aggregation for each generated metric in the search space
* feature selection from the aggregated metrics
* fitting of the 1-st level meta-estimators on the selected features
* aggregation function optimization over predictions of the 1-st level meta-estimators

In [ ]:
q_clf.fit(X, Xy, y)

In [ ]:
# predictions of the meta-classifier on test data:
y_pred = q_clf.predict(X_test, Xy_test)

In [ ]:
# example of automatically selected features:
print(q_clf.selected_features['rf'])

## Submission preparation:

In [ ]:
root_dir = "../data/decomposed/"

SecretPart_427 = prepare_submission(y_pred, root_dir)

In [ ]:
SecretPart_427.head(3)

In [ ]:
SecretPart_427.to_csv("../data/SecretPart_427.csv", index=False)